In [1]:
import os
os.environ["EURI_API_KEY"] = "euri-e4ac18a3dd84d7880024dceca11b980a88964d6e7faa2ed7c854d0ab6fea2bfd"

In [2]:
# ================================
# Namma Metro RAG Demo (with 50% cutoff message)
# ================================
# REQUIREMENTS:
#   pip install pandas numpy nltk requests faiss-cpu
#
# IMPORTANT: Do NOT hardcode your key in code.
#   import os; os.environ["EURI_API_KEY"] = "YOUR_KEY"
# ================================

import os
import json
import re
import string
import unicodedata
from typing import List, Dict, Tuple, Any

import pandas as pd
import numpy as np
import requests

# --- Read API key from env (preferred). ---
EURI_API_KEY = os.getenv("EURI_API_KEY", "")
if not EURI_API_KEY:
    raise ValueError("EURI_API_KEY environment variable not set. Please set it to your API key.")


In [ ]:
# -----------------------------
# 1) Dataset (original text)
# -----------------------------
import os
dir_path = "C:\Data Science\Assignments\Final\metro_rag_application\data"
file_path = os.path.join(dir_path, "dataset.txt")

with open(file_path, "r", encoding="utf-8") as f:
    dataset = f.read()

In [ ]:

dataset = """Namma Metro: Expanding Horizons and Navigating Challenges
1. A Heroic Moment—Fire Averted at Hebbagodi

On the night of August 23, 2025, a crisis was averted at the newly inaugurated Hebbagodi Metro station on the Yellow Line. Around 11:50 PM, a car stalled beneath the elevated station on the waterlogged Hosur Road. Smoke began billowing from the engine bay—a potential fire hazard. That’s when Shivakumar, the on-duty security guard, sprang into action. He grabbed a fire extinguisher from inside the station and swiftly doused the flames, likely caused by a short circuit. His quick response prevented damage to metro infrastructure and avoided additional traffic disruption. Officials praised the effectiveness of routine fire safety training in empowering such timely action. 
Hindustan Times
This incident underscores the vulnerability of metro infrastructure to external hazards and highlights how preparedness and vigilance—especially at newly operational stations—are crucial in maintaining safety.

2. Managing Crush: Barricades at RV Road Station

The opening of the Yellow Line has significantly boosted ridership, particularly at RV Road interchange, where passengers transfer to the Green Line or head toward Electronics City. The daily interchange volume has surged to an estimated 30,000–35,000 passengers. To manage the resulting crowding, BMRCL has installed barricades—marking the first time such measures have been installed on an elevated station (previously only at the underground Majestic interchange). 
Currently, Yellow Line services run every 25 minutes with three trains, but a fourth train set is expected by mid-September after assembly and testing—anticipated to reduce headways to around 20 minutes. 
3. Yellow Line Extension Study to Attibele

Connectivity continues to be a priority. The Karnataka government, led by Deputy CM D.K. Shivakumar, has commissioned a feasibility study to explore extending the Yellow Line beyond Bommasandra to Attibele, spanning over 10 km. This expansion is aimed at reducing congestion along Hosur Road and supporting proposed infrastructure developments, including a new stadium project in Attibele. 
The study has been entrusted to Hyderabad-based RV Associates, tasked with preparing a Detailed Project Report (DPR) that evaluates costs, benefits, and technical feasibility. 
4. Phase 3 Expansion: Environmental Challenges Delay Progress
Meanwhile, Phase 3 of the metro expansion, encompassing a 44-km fully elevated network split between corridors from JP Nagar 4th Phase to Kempapura and Hosahalli to Kadabagere, has hit a regulatory roadblock. The Environmental Impact Assessment (EIA) report has been delayed, as the initial round of public stakeholder meetings saw low turnout. This necessitated additional public consultation and pushed back both the EIA and the final DPR. 
Tree impact estimates have shifted—from an initial upper bound of 11,000 trees to a revised count of around 6,000, though the final figure remains pending the completed EIA. The BMRCL has committed to tree transplantation or sapling replantation, though clarity is required about replantation locations. Critics note that proposed replantation in areas like Kolar, far from impacted zones, may not align with ecological restoration goals. 
The Times of India
5. Early Morning Yellow Line Boost on August 18

To manage the surge of commuters following a long Independence Day weekend, BMRCL advanced Yellow Line train services to start at 5 AM on August 18, instead of the regular 6:30 AM schedule. The early service began from both RV Road and Delta Electronics Bommasandra stations. This adjustment was temporary for that Monday; normal timings resumed by Tuesday thereafter. 
Such responsiveness signals the operational flexibility BMRCL is building into its service planning to adapt to commuter patterns.
6. A Game-Changer: Yellow Line Inaugurated
The much-anticipated Yellow Line, stretching 19.15 kilometers between RV Road and Bommasandra, officially opened to the public in early August 2025, with Prime Minister Narendra Modi inaugurating the service. 
Constructed at a cost of approximately ₹7,160–7,610 crore, this elevated corridor features 16 stations and aims to support up to 800,000 daily commuters. 
The Economic Times
 The line traverses key junctions including Jayadeva, BTM Layout, Central Silk Board, and Bommasandra, significantly enhancing connectivity to important residential, commercial, and industrial zones. 
The introduction of the Yellow Line complements existing network routes, enabling faster, more seamless interchanges and contributing to overall network efficiency. Commuters are expected to enjoy improved convenience, travel speed, and comfort. 
The Economic Times
7. Last-Mile Concerns: Double-Decker Flyover Funding Tussle
While the Yellow Line has been a massive lift for metro connectivity, financial hurdles persist. BMRCL is still awaiting ₹450 crore in reimbursement from the state government, promised back in 2017–18, to cover costs associated with a double-decker flyover between Ragigudda and Central Silk Board. A 5 km section of this flyover opened in July 2024, but the reimbursement is still pending. 
The Times of India
The state government is now pushing forward with plans for another 45 km of double-decker flyovers under Phase 3—budgeted at ₹15,611 crore, with roughly ₹9,000 crore allocated for road flyover construction. Deputy CM D.K. Shivakumar has sought ₹8,916 crore in central funding. However, urban planners warn that such flyovers might encourage private vehicle usage, undermining public transportation goals. 
Broader Context: Namma Metro at a Glance
As of August 2025, Namma Metro spans roughly 96 km, serving 83–85 operational stations (estimates vary slightly across sources). It transports around 762,361 average passengers daily (data from July 2024), with a single-day record of 1,048,031 riders set on August 11, 2025, just after Yellow Line became operational. 
Wikipedia
Ongoing and Future Lines
Pink Line: A 21.25 km route from Kalena Agrahara to Nagawara, including a 7.5 km elevated section (to open by March 2026) and a 13.76 km underground corridor (via MG Road to Dairy Circle–Nagavara) slated to open by September 2026. 
Blue Line (Airport Link): Intended to connect Central Silk Board with Kempegowda International Airport via KR Pura and Nagawara. The entire line is expected to be ready by June 2026. 
Phase 3 Corridors:
Orange Line (ORR-West): Kempapura to JP Nagar 4th Phase, approved and expected by 2029. 
Grey Line (Hosahalli–Kadabagere): Also approved, with a target completion around 2029. 
Red Line (Hebbal–Sarjapura): Part of Phase 3 projects, with DPR already in progress. 
Fare Hike Fallout: In early 2025, Namma Metro implemented steep fare increases, some routes witnessing over 100% hikes (later capped at around 71%). Ridership declined and public backlash ensued. 
Final Thoughts: A Transit Revolution in Motion
Bengaluru’s metro network is undergoing a dramatic transformation. The newly opened Yellow Line boosts coverage in southern Bengaluru and offers critical interchange points—network synergies that promise smoother travel. Crowd management innovations like barricades at RV Road and earlier operational flexibility (e.g., August 18 early runs) show how responsive BMRCL is adapting to real-world demands.
Yet, challenges remain. Securing funding for infrastructure like flyovers, navigating environmental clearances for Phase 3, and making extensions like Attibele feasible—all require careful planning, sustainable funding, and stakeholder coordination.
Looking ahead, integration of upcoming Pink and Blue lines, the completion of Phase 3 corridors (Orange, Grey, Red), and seamless operational integration will define the next frontier. If executed well, Namma Metro can help Bengaluru mitigate congestion, promote equitable mobility, and reinforce its standing as one of India’s most forward-looking urban transit networks.
"""

In [4]:
# Optional: save dataset to CSV
pd.DataFrame({"text": [dataset]}).to_csv("namma_metro_dataset.csv", index=False)

In [5]:
# -----------------------------
# 2) Text cleaning
# -----------------------------
import nltk
nltk.download("punkt", quiet=True)
nltk.download("stopwords", quiet=True)
nltk.download("wordnet", quiet=True)
nltk.download("omw-1.4", quiet=True)

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

stop_words = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()

def clean_text(
    text: str,
    lower: bool = True,
    remove_punct: bool = True,
    remove_stop: bool = True,
    lemmatize_tokens: bool = True,
    remove_numbers: bool = True,
    keep_only_alpha: bool = True
):
    text = unicodedata.normalize("NFKD", text)
    if lower:
        text = text.lower()
    if remove_punct:
        text = text.translate(str.maketrans("", "", string.punctuation))
    tokens = nltk.word_tokenize(text)

    cleaned_tokens: List[str] = []
    for token in tokens:
        if remove_numbers and token.isdigit():
            continue
        if keep_only_alpha and not token.isalpha():
            continue
        if remove_stop and token in stop_words:
            continue
        if lemmatize_tokens:
            token = lemmatizer.lemmatize(token)
        cleaned_tokens.append(token)
    return " ".join(cleaned_tokens)

cleaned_data = clean_text(dataset)

print(f"Cleaned length: {len(cleaned_data)}")
print(f"Clean data: {cleaned_data}")



Cleaned length: 5990
Clean data: namma metro expanding horizon navigating challenge heroic averted hebbagodi night august crisis averted newly inaugurated hebbagodi metro station yellow line around pm car stalled beneath elevated station waterlogged hosur road smoke began billowing engine potential fire hazard shivakumar onduty security guard sprang action grabbed fire extinguisher inside station swiftly doused flame likely caused short circuit quick response prevented damage metro infrastructure avoided additional traffic disruption official praised effectiveness routine fire safety training empowering timely action hindustan time incident underscore vulnerability metro infrastructure external hazard highlight preparedness newly operational crucial maintaining safety managing crush barricade rv road station opening yellow line significantly boosted ridership particularly rv road interchange passenger transfer green line head toward electronics city daily interchange volume surged esti

In [6]:
# -----------------------------
# 3) Chunking
# -----------------------------
def chunk_text(text: str, max_char: int = 500, overlap: int = 100) -> List[str]:
    assert max_char > overlap >= 0, "max_char must be > overlap >= 0"
    chunks: List[str] = []
    i = 0
    n = len(text)
    while i < n:
        piece = text[i : i + max_char]
        chunks.append(piece)
        i += max_char - overlap
    return chunks

In [7]:
chunks = chunk_text(cleaned_data, max_char=500, overlap=100)
print(f"Total chunks created: {len(chunks)}")
print(f"chunk preview: {chunks}")

Total chunks created: 15
chunk preview: ['namma metro expanding horizon navigating challenge heroic averted hebbagodi night august crisis averted newly inaugurated hebbagodi metro station yellow line around pm car stalled beneath elevated station waterlogged hosur road smoke began billowing engine potential fire hazard shivakumar onduty security guard sprang action grabbed fire extinguisher inside station swiftly doused flame likely caused short circuit quick response prevented damage metro infrastructure avoided additional traffic disr', 'd short circuit quick response prevented damage metro infrastructure avoided additional traffic disruption official praised effectiveness routine fire safety training empowering timely action hindustan time incident underscore vulnerability metro infrastructure external hazard highlight preparedness newly operational crucial maintaining safety managing crush barricade rv road station opening yellow line significantly boosted ridership particularly rv

In [8]:
# 4) Embeddings via Euron API
#now generate embeddings for each chunk using euron api
import requests
import numpy as np

def generate_embeddings(text):
    url = "https://api.euron.one/api/v1/euri/embeddings"
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {EURI_API_KEY}"
    }
    payload = {
        "input": text,
        "model": "text-embedding-3-small"
    }

    response = requests.post(url, headers=headers, json=payload)
    data = response.json()
    
    embedding = np.array(data['data'][0]['embedding'])
    
    return embedding

In [9]:
def create_embeddings(all_chunks: List[str]) -> Tuple[List[np.ndarray], List[Dict[str, Any]]]:
    emb_list: List[np.ndarray] = []
    meta: List[Dict[str, Any]] = []
    for idx, chunk in enumerate(all_chunks):
        vec = generate_embeddings(chunk)
        emb_list.append(vec.astype("float32"))
        meta.append({"id": idx, "text": chunk})
    return emb_list, meta

emb_list, meta = create_embeddings(chunks)

In [10]:
emb_list

[array([-0.00054488, -0.00683253,  0.03200348, ..., -0.0087699 ,
         0.02067507, -0.00800087], dtype=float32),
 array([-0.00293541, -0.01590907,  0.02849055, ..., -0.01880883,
         0.00452791,  0.0176521 ], dtype=float32),
 array([-0.01344002, -0.01362632,  0.06988812, ..., -0.00288927,
        -0.02620139,  0.0101399 ], dtype=float32),
 array([ 0.01760913, -0.03226775,  0.077625  , ...,  0.0048415 ,
         0.01487321,  0.01163436], dtype=float32),
 array([ 0.02456949, -0.03474645,  0.06801675, ...,  0.01290218,
         0.01477576,  0.01415833], dtype=float32),
 array([0.04711136, 0.02026819, 0.06898151, ..., 0.01331711, 0.0364131 ,
        0.02228805], dtype=float32),
 array([0.02615949, 0.03281878, 0.07771981, ..., 0.00222503, 0.0112955 ,
        0.00251831], dtype=float32),
 array([ 0.01034963,  0.00972444,  0.07453366, ..., -0.0019877 ,
         0.00478067,  0.00283035], dtype=float32),
 array([ 0.00528324, -0.05408125,  0.06017135, ...,  0.01058752,
        -0.00328711

In [11]:
meta

[{'id': 0,
  'text': 'namma metro expanding horizon navigating challenge heroic averted hebbagodi night august crisis averted newly inaugurated hebbagodi metro station yellow line around pm car stalled beneath elevated station waterlogged hosur road smoke began billowing engine potential fire hazard shivakumar onduty security guard sprang action grabbed fire extinguisher inside station swiftly doused flame likely caused short circuit quick response prevented damage metro infrastructure avoided additional traffic disr'},
 {'id': 1,
  'text': 'd short circuit quick response prevented damage metro infrastructure avoided additional traffic disruption official praised effectiveness routine fire safety training empowering timely action hindustan time incident underscore vulnerability metro infrastructure external hazard highlight preparedness newly operational crucial maintaining safety managing crush barricade rv road station opening yellow line significantly boosted ridership particularly 

In [12]:
# 5) FAISS index save/load (cosine via IP)
# -----------------------------
import faiss

def save_faiss_index(
    emb_list: List[np.ndarray],
    meta: List[Dict[str, Any]],
    index_path: str = "index_vecass1.faiss",
    meta_path: str = "meta_vecass1.jsonl"
) -> None:
    xb = np.vstack(emb_list).astype("float32")
    faiss.normalize_L2(xb)
    dim = xb.shape[1]
    index = faiss.IndexFlatIP(dim)
    index.add(xb)
    faiss.write_index(index, index_path)

    with open(meta_path, "w", encoding="utf-8") as f:
        for item in meta:
            f.write(json.dumps(item, ensure_ascii=False) + "\n")

    print(f"FAISS index saved -> {index_path}")
    print(f"Metadata saved   -> {meta_path} (records: {len(meta)})")

In [13]:
save_faiss_index(emb_list, meta, "index_vecass1.faiss", "meta_vecass1.jsonl")

FAISS index saved -> index_vecass1.faiss
Metadata saved   -> meta_vecass1.jsonl (records: 15)


In [ ]:
# Load FAISS index and metadata
from typing import Tuple, List, Dict, Any

def load_faiss_index(index_path: str, meta_path: str):
    index = faiss.read_index(index_path)
    meta_list: List[Dict[str, Any]] = []
    with open(meta_path, "r", encoding="utf-8") as f:
        for line in f:
            meta_list.append(json.loads(line))
    print(f"Loaded index: {index_path} | metadata records: {len(meta_list)}")
    return index, meta_list




Loaded index: index_vecass1.faiss | metadata records: 15


In [25]:
index, meta = load_faiss_index("index_vecass1.faiss", "meta_vecass1.jsonl")

Loaded index: index_vecass1.faiss | metadata records: 15


In [26]:
index

<faiss.swigfaiss_avx2.IndexFlatIP; proxy of <Swig Object of type 'faiss::IndexFlatIP *' at 0x0000020A50FF2220> >

In [27]:
meta

[{'id': 0,
  'text': 'namma metro expanding horizon navigating challenge heroic averted hebbagodi night august crisis averted newly inaugurated hebbagodi metro station yellow line around pm car stalled beneath elevated station waterlogged hosur road smoke began billowing engine potential fire hazard shivakumar onduty security guard sprang action grabbed fire extinguisher inside station swiftly doused flame likely caused short circuit quick response prevented damage metro infrastructure avoided additional traffic disr'},
 {'id': 1,
  'text': 'd short circuit quick response prevented damage metro infrastructure avoided additional traffic disruption official praised effectiveness routine fire safety training empowering timely action hindustan time incident underscore vulnerability metro infrastructure external hazard highlight preparedness newly operational crucial maintaining safety managing crush barricade rv road station opening yellow line significantly boosted ridership particularly 

In [29]:
# 6) Search + threshold logic
# -----------------------------
THRESHOLD = 0.50  # 50%

def search_faiss(index: faiss.IndexFlatIP, query: str, top_k: int = 6):
    q = generate_embeddings(query).reshape(1, -1).astype("float32")
    faiss.normalize_L2(q)
    scores, idxs = index.search(q, top_k)
    return scores, idxs

In [30]:
def results_to_paragraph(
    data_output: Tuple[np.ndarray, np.ndarray],
    meta: List[Dict[str, Any]],
    threshold: float = THRESHOLD
):
    scores, idxs = data_output
    scores = np.asarray(scores).ravel().astype(float)
    idxs = np.asarray(idxs).ravel().astype(int)

    meta_by_id = {int(m["id"]): m["text"] for m in meta}

    selected: List[str] = []
    for i, s in zip(idxs, scores):
        if s >= threshold and i in meta_by_id:
            selected.append(meta_by_id[i])

    # Custom message per your requirement:
    if not selected:
        return "I am not trend on given data"

    return " ".join(selected)

In [31]:
def retrieve_context(
    data_output: Tuple[np.ndarray, np.ndarray],
    meta: List[Dict[str, Any]],
    top_n: int = 3,
    threshold: float = THRESHOLD
):
    scores, idxs = data_output
    scores = np.asarray(scores).ravel().astype(float)
    idxs = np.asarray(idxs).ravel().astype(int)

    meta_by_id = {int(m["id"]): m["text"] for m in meta}

    pairs = [(i, s) for i, s in zip(idxs, scores) if s >= threshold and i in meta_by_id]
    pairs = pairs[:top_n]

    return [(meta_by_id[i], float(s)) for i, s in pairs]

In [32]:

# 7) Prompt + completion (only if threshold passed)

def generate_completion(prompt, model="gpt-4.1-nano"):
    url = "https://api.euron.one/api/v1/euri/chat/completions"
    headers = {
        "Authorization": f"Bearer {EURI_API_KEY}",
        "Content-Type": "application/json"
    }
    payload = {
        "model": model,
        "messages": [{"role": "user", "content": prompt}],
        "max_tokens": 500,
        "temperature": 0.3
    }
    response = requests.post(url, headers=headers, json=payload)
    return response.json()['choices'][0]['message']['content']
	
	


In [33]:
def build_prompt(query: str, context_chunks: List[Tuple[str, float]]) -> str:
    context_texts = "\n\n".join([c[0] for c in context_chunks])
    prompt = f"""You are a helpful assistant. 
Here is some context from the knowledge base:

{context_texts}

Based on the above, answer this question:

{query}
"""
    return prompt

In [36]:
# 8) Demo run
# -----------------------------
query = "double-decker flyovers under Phase 3—budgeted"

data_output = search_faiss(index, query, top_k=6)

# First, check the thresholded paragraph:
paragraph = results_to_paragraph(data_output, meta, threshold=THRESHOLD)
print("\n--- Thresholded Paragraph or Message ---\n", paragraph, "\n")
if paragraph.strip() == "I am not trend on given data":
    # Nothing passed the 50% threshold; stop here as requested.
    pass
else:
    # We have relevant context; proceed to build a prompt and call the model.
    context_chunks = retrieve_context(data_output, meta, top_n=3, threshold=THRESHOLD)
    # Safety: if, for some reason, context_chunks is empty, also show the same message.
    if not context_chunks:
        print("I am not trend on given data")
    else:
        prompt = build_prompt(query, context_chunks)
        print("\n--- Prompt Sent to Model ---\n", prompt, "\n")
        answer = generate_completion(prompt)
        print("--- Model Answer ---\n", answer)


--- Thresholded Paragraph or Message ---
 ledecker flyover funding tussle yellow line massive lift metro connectivity financial hurdle persist bmrcl still awaiting crore reimbursement state government promised back cover cost associated doubledecker flyover ragigudda central silk board km section flyover opened july reimbursement still pending time india state government pushing forward plan another km doubledecker flyover phase crore roughly crore allocated road flyover construction deputy cm dk shivakumar sought crore central funding  ghly crore allocated road flyover construction deputy cm dk shivakumar sought crore central funding however urban planner warn flyover might encourage private vehicle usage undermining public transportation goal broader context namma metro glance august namma metro span roughly km serving operational station estimate vary slightly across source transport around average passenger daily data july singleday record rider set august yellow line became opera

In [39]:
content

'1. A Heroic Moment—Fire Averted at Hebbagodi\n\nOn the night of August 23, 2025, a crisis was averted at the newly inaugurated Hebbagodi Metro station on the Yellow Line. Around 11:50 PM, a car stalled beneath the elevated station on the waterlogged Hosur Road. Smoke began billowing from the engine bay—a potential fire hazard. That’s when Shivakumar, the on-duty security guard, sprang into action. He grabbed a fire extinguisher from inside the station and swiftly doused the flames, likely caused by a short circuit. His quick response prevented damage to metro infrastructure and avoided additional traffic disruption. Officials praised the effectiveness of routine fire safety training in empowering such timely action. \nHindustan Times\nThis incident underscores the vulnerability of metro infrastructure to external hazards and highlights how preparedness and vigilance—especially at newly operational stations—are crucial in maintaining safety.\n\n2. Managing Crush: Barricades at RV Road 

In [37]:
print(os.getcwd())

c:\Data Science\Assignments\Final
